In [16]:
import pandas as pd

animes = pd.read_csv("animes_with_genres.csv")

In [3]:
from transformers import pipeline
classifier = pipeline("text-classification",
                      model="j-hartmann/emotion-english-distilroberta-base",
                      top_k = None,
                      device = 0)
classifier("I love this!")


Device set to use cuda:0


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.008528684265911579},
  {'label': 'neutral', 'score': 0.005764597095549107},
  {'label': 'anger', 'score': 0.004419783595949411},
  {'label': 'sadness', 'score': 0.002092392183840275},
  {'label': 'disgust', 'score': 0.001611993182450533},
  {'label': 'fear', 'score': 0.0004138521908316761}]]

In [6]:
animes["description"][0]

"Crime is timeless. By the year 2071, humanity has expanded across the galaxy, filling the surface of other planets with settlements like those on Earth. These new societies are plagued by murder, drug use, and theft, and intergalactic outlaws are hunted by a growing number of tough bounty hunters.Spike Spiegel and Jet Black pursue criminals throughout space to make a humble living. Beneath his goofy and aloof demeanor, Spike is haunted by the weight of his violent past. Meanwhile, Jet manages his own troubled memories while taking care of Spike and the Bebop, their ship. The duo is joined by the beautiful con artist Faye Valentine, odd child Edward Wong Hau Pepelu Tivrusky IV, and Ein, a bioengineered Welsh corgi.While developing bonds and working to catch a colorful cast of criminals, the Bebop crew's lives are disrupted by a menace from Spike's past. As a rival's maniacal plot continues to unravel, Spike must choose between life with his newfound family or revenge for his old wounds

In [7]:
classifier(animes["description"][0])

[[{'label': 'fear', 'score': 0.7629051208496094},
  {'label': 'sadness', 'score': 0.1202375739812851},
  {'label': 'neutral', 'score': 0.056413888931274414},
  {'label': 'disgust', 'score': 0.03398098051548004},
  {'label': 'anger', 'score': 0.020432112738490105},
  {'label': 'surprise', 'score': 0.003134337952360511},
  {'label': 'joy', 'score': 0.002895953832194209}]]

In [8]:
classifier(animes["description"][0].split("."))

[[{'label': 'fear', 'score': 0.5208521485328674},
  {'label': 'neutral', 'score': 0.19815345108509064},
  {'label': 'anger', 'score': 0.16256728768348694},
  {'label': 'disgust', 'score': 0.0578363761305809},
  {'label': 'sadness', 'score': 0.05312264338135719},
  {'label': 'surprise', 'score': 0.004628059454262257},
  {'label': 'joy', 'score': 0.0028400158043950796}],
 [{'label': 'neutral', 'score': 0.8242284655570984},
  {'label': 'surprise', 'score': 0.07020925730466843},
  {'label': 'fear', 'score': 0.03644148260354996},
  {'label': 'anger', 'score': 0.026523057371377945},
  {'label': 'disgust', 'score': 0.025757666677236557},
  {'label': 'sadness', 'score': 0.009884167462587357},
  {'label': 'joy', 'score': 0.0069558932445943356}],
 [{'label': 'fear', 'score': 0.6876932978630066},
  {'label': 'anger', 'score': 0.12076738476753235},
  {'label': 'neutral', 'score': 0.10485956817865372},
  {'label': 'disgust', 'score': 0.05930459499359131},
  {'label': 'sadness', 'score': 0.020458174

In [9]:
sentences = animes["description"][0].split(".")
predictions = classifier(sentences)

In [10]:
sentences[0]

'Crime is timeless'

In [11]:
predictions[0]

[{'label': 'fear', 'score': 0.5208521485328674},
 {'label': 'neutral', 'score': 0.19815345108509064},
 {'label': 'anger', 'score': 0.16256728768348694},
 {'label': 'disgust', 'score': 0.0578363761305809},
 {'label': 'sadness', 'score': 0.05312264338135719},
 {'label': 'surprise', 'score': 0.004628059454262257},
 {'label': 'joy', 'score': 0.0028400158043950796}]

In [12]:
sorted(predictions[0], key=lambda x: x["label"])

[{'label': 'anger', 'score': 0.16256728768348694},
 {'label': 'disgust', 'score': 0.0578363761305809},
 {'label': 'fear', 'score': 0.5208521485328674},
 {'label': 'joy', 'score': 0.0028400158043950796},
 {'label': 'neutral', 'score': 0.19815345108509064},
 {'label': 'sadness', 'score': 0.05312264338135719},
 {'label': 'surprise', 'score': 0.004628059454262257}]

In [13]:
import numpy as np

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
ids = []
emotion_scores = {label: [] for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x: x["label"])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])

    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}

In [22]:
from tqdm import tqdm

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
ids = []
emotion_scores = {label: [] for label in emotion_labels}
n = len(animes) // 8
print(n)

for i in tqdm(range(n)):
    ids.append(animes["myanimelist_id"][i])
    sentences = animes["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

725


100%|██████████| 725/725 [00:48<00:00, 14.99it/s]


In [25]:
emotion_scores = {
    key: [float(v) for v in values]
    for key, values in emotion_scores.items()
}

emotions_df = pd.DataFrame(emotion_scores)
emotions_df["myanimelist_id"] = ids

In [27]:
emotions_df

,anger,disgust,fear,joy,sadness,surprise,neutral,myanimelist_id
0,0.687464,0.075986,0.958079,0.234636,0.947046,0.871028,0.127862,1
1,0.687464,0.075986,0.958079,0.234636,0.947046,0.871028,0.127862,1
2,0.687464,0.075986,0.958079,0.234636,0.947046,0.871028,0.127862,1
3,0.842108,0.309940,0.395538,0.026244,0.947046,0.238725,0.032194,6
4,0.842108,0.309940,0.395538,0.026244,0.947046,0.238725,0.032194,6
...,...,...,...,...,...,...,...,...
720,0.031158,0.972955,0.023587,0.175381,0.916754,0.957265,0.156560,386
721,0.031158,0.972955,0.023587,0.175381,0.916754,0.957265,0.156560,386
722,0.051169,0.120043,0.885468,0.097567,0.947046,0.201337,0.429503,387
723,0.051169,0.120043,0.885468,0.097567,0.947046,0.201337,0.429503,387


In [28]:
animes = pd.merge(animes, emotions_df, on="myanimelist_id")

In [29]:
animes

,myanimelist_id,title,description,image,Episodes,Status,Premiered,Released_Season,Released_Year,Source,...,source_url,tagged_description,simple_genres,anger,disgust,fear,joy,sadness,surprise,neutral
0,1,Cowboy Bebop,"Crime is timeless. By the year 2071, humanity ...",https://cdn.myanimelist.net/images/anime/4/196...,26.0,Finished Airing,Spring 1998,Spring,1998.0,Original,...,https://myanimelist.net/anime/1/Cowboy_Bebop,"1 Crime is timeless. By the year 2071, humanit...",Realistic,0.687464,0.075986,0.958079,0.234636,0.947046,0.871028,0.127862
1,1,Cowboy Bebop,"Crime is timeless. By the year 2071, humanity ...",https://cdn.myanimelist.net/images/anime/4/196...,26.0,Finished Airing,Spring 1998,Spring,1998.0,Original,...,https://myanimelist.net/anime/1/Cowboy_Bebop,"1 Crime is timeless. By the year 2071, humanit...",Realistic,0.687464,0.075986,0.958079,0.234636,0.947046,0.871028,0.127862
2,1,Cowboy Bebop,"Crime is timeless. By the year 2071, humanity ...",https://cdn.myanimelist.net/images/anime/4/196...,26.0,Finished Airing,Spring 1998,Spring,1998.0,Original,...,https://myanimelist.net/anime/1/Cowboy_Bebop,"1 Crime is timeless. By the year 2071, humanit...",Realistic,0.687464,0.075986,0.958079,0.234636,0.947046,0.871028,0.127862
3,1,Cowboy Bebop,"Crime is timeless. By the year 2071, humanity ...",https://cdn.myanimelist.net/images/anime/4/196...,26.0,Finished Airing,Spring 1998,Spring,1998.0,Original,...,https://myanimelist.net/anime/1/Cowboy_Bebop,"1 Crime is timeless. By the year 2071, humanit...",['Realistic'],0.687464,0.075986,0.958079,0.234636,0.947046,0.871028,0.127862
4,1,Cowboy Bebop,"Crime is timeless. By the year 2071, humanity ...",https://cdn.myanimelist.net/images/anime/4/196...,26.0,Finished Airing,Spring 1998,Spring,1998.0,Original,...,https://myanimelist.net/anime/1/Cowboy_Bebop,"1 Crime is timeless. By the year 2071, humanit...",['Realistic'],0.687464,0.075986,0.958079,0.234636,0.947046,0.871028,0.127862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2112,387,Haibane Renmei,"Born from a cocoon in the village of Old Home,...",https://cdn.myanimelist.net/images/anime/9/131...,13.0,Finished Airing,Fall 2002,Fall,2002.0,Other,...,https://myanimelist.net/anime/387/Haibane_Renmei,387 Born from a cocoon in the village of Old H...,['Realistic'],0.051169,0.120043,0.885468,0.097567,0.947046,0.201337,0.429503
2113,387,Haibane Renmei,"Born from a cocoon in the village of Old Home,...",https://cdn.myanimelist.net/images/anime/9/131...,13.0,Finished Airing,Fall 2002,Fall,2002.0,Other,...,https://myanimelist.net/anime/387/Haibane_Renmei,387 Born from a cocoon in the village of Old H...,['Realistic'],0.051169,0.120043,0.885468,0.097567,0.947046,0.201337,0.429503
2114,387,Haibane Renmei,"Born from a cocoon in the village of Old Home,...",https://cdn.myanimelist.net/images/anime/9/131...,13.0,Finished Airing,Fall 2002,Fall,2002.0,Other,...,https://myanimelist.net/anime/387/Haibane_Renmei,387 Born from a cocoon in the village of Old H...,Realistic,0.051169,0.120043,0.885468,0.097567,0.947046,0.201337,0.429503
2115,387,Haibane Renmei,"Born from a cocoon in the village of Old Home,...",https://cdn.myanimelist.net/images/anime/9/131...,13.0,Finished Airing,Fall 2002,Fall,2002.0,Other,...,https://myanimelist.net/anime/387/Haibane_Renmei,387 Born from a cocoon in the village of Old H...,Realistic,0.051169,0.120043,0.885468,0.097567,0.947046,0.201337,0.429503


In [30]:
animes.to_csv("animes_with_emotions.csv")